### Challenge 1

In [3]:
# challenge 1

# instructions are unclear...

import base64
from base64 import b64encode 
import binascii

# Convert hex to base64
# indicates this string to be converted to base64
myHex = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
targetBase64 = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


# This is not "a string"; it is a hex-encoded string.  I thought it was a string representing hex data.

# So insted we decode the string and convert that string to base64

# parse hex string into sequence of bytes
decoded_str_bytes = bytes.fromhex(myHex)
print(f'{decoded_str_bytes=}')
print(f'{decoded_str_bytes.decode("utf-8")=}')

b64 = b64encode(decoded_str_bytes)
print(f'{b64=}')

base64string = b64.decode('utf-8')
print(f'{base64string=}')

assert targetBase64 == base64string, base64string


decoded_str_bytes=b"I'm killing your brain like a poisonous mushroom"
decoded_str_bytes.decode("utf-8")="I'm killing your brain like a poisonous mushroom"
b64=b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'
base64string='SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


### Challenge 2

In [30]:
# challenge 2
# this time what the challenge calls a string is actually a hex value.

# string to int type conversions
s1 = int('1c0111001f010100061a024b53535009181c', 16)
s2 = int('686974207468652062756c6c277320657965', 16)

target = '746865206b696420646f6e277420706c6179'

# do the xor, convert int to string
candidate = f'{s1^s2:x}'

assert target == candidate, candidate

# treat hex values as ascii values and map to characters
bytes.fromhex(target).decode('utf-8')


"the kid don't play"

### Challenge 3

In [108]:
# from the start, the instructions are much more precise
encrypted = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
encrypted_bytes = bytes.fromhex(encrypted)
num_bytes = len(encrypted_bytes)
print(f'{num_bytes=}')
encrypted_hex = int(encrypted, 16)
print(encrypted_bytes)

num_bytes=34
b'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'


In [103]:
from collections import Counter
import string

vocab = string.ascii_lowercase + ' '# + string.ascii_uppercase
def english_score(txt):
    counter = Counter(txt)
    
    # txt gets a point for common alphabetic characters
    most = counter.most_common()
    score = sum(1 if tup[0] in vocab else -1 for tup in most)
    return score

def english_score2(txt):
    counter = Counter(txt)
    
    # txt gets a point for common alphabetic characters
    most = counter.most_common()
    score = sum(val*tup[1] if tup[0] in vocab else -val*tup[1] 
                for tup, val in zip(most,range(len(most),1,-1)))
    return score

english_score2('aaa;')

6

In [240]:

def decrypt_using_single_byte(encrypted_txt):
    encrypted_bytes = bytes.fromhex(encrypted_txt)
    num_bytes = len(encrypted_bytes)
    #print(f'{num_bytes=}')
    encrypted_hex = int(encrypted_txt, 16)
    #print(encrypted_bytes)

    scores = []
    scores2 = []
    
    for single_byte in range(0,256):
        #print()
        #print(f'{single_byte=:x}')


        ######
        # This doesn't work because UTF-8 encoding becomes
        # two bytes for codes above 127
        # turn string to bytes, to hex string, to int
        ######
        #key = chr(single_byte)*num_bytes
        #print(f'{key=}')
        #keyHex = int(key.encode('utf-8').hex(), 16)

        ######
        # Instead, construct the bytes directly
        ######
        key = [single_byte] * num_bytes
        keyHex = int(bytes(key).hex(), 16)


        #print(f'{keyHex=}')
        #print(f'{len(str(keyHex))=}')

        decrypted_hex = encrypted_hex ^ keyHex
        #print(decrypted_hex)

        # ensure the length of the hex string matches the length of the
        # encrypted bytes, otherwise leading 0s get dropped and cause exceptions
        decrypted_hex_str = f'{decrypted_hex:0{num_bytes*2}x}'
        #print(f'{decrypted_hex_str=}')
        #print(f'{len(decrypted_hex_str)=}')

        ######
        # For the same reason above, converting to a hex string and
        # decoding as utf-8 doesn't work
        ######
        #ptxt = bytes.fromhex(decrypted_hex_str).decode('utf-8')


        ptxt_bytes = bytes.fromhex(decrypted_hex_str)
        ptxt = ''
        for b in ptxt_bytes:
            ptxt += chr(b)
        #print(f'{ptxt=}')


        scores.append(
            (ptxt, english_score(ptxt), single_byte)
        )
        scores2.append(
            (ptxt, english_score2(ptxt), single_byte)
        )
        
    def sorter(i):
        return i[1]
    scores.sort(key=sorter, reverse=True)
    scores2.sort(key=sorter, reverse=True)

    return scores[0], scores, scores2

In [241]:
encrypted = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
_, scores, scores2 = decrypt_using_single_byte(encrypted)

for tup in scores[:10]:
    print(tup)
print()
for tup in scores2[:10]:
    print(tup)
    

("Cooking MC's like a pound of bacon", 13, 88)
("Dhhlni`'JD t'knlb'f'whric'ha'efdhi", 11, 95)
('Ieeacdm*GI-y*fcao*k*ze\x7fdn*el*hkied', 9, 82)
('Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno', 9, 89)
('Ammikle"OA%q"nkig"c"rmwlf"md"`caml', 9, 90)
('@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm', 9, 91)
('Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj', 9, 92)
('Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk', 9, 93)
('Eiimoha&KE!u&jomc&g&vishb&i`&dgeih', 9, 94)
('Uyy}\x7fxq6[U1e6z\x7f}s6w6fycxr6yp6twuyx', 7, 78)

("Cooking MC's like a pound of bacon", 342, 88)
('@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm', 114, 91)
('Ammikle"OA%q"nkig"c"rmwlf"md"`caml', 110, 90)
("Dhhlni`'JD t'knlb'f'whric'ha'efdhi", 110, 95)
('Eiimoha&KE!u&jomc&g&vishb&i`&dgeih', 108, 94)
('Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj', 106, 92)
('Ieeacdm*GI-y*fcao*k*ze\x7fdn*el*hkied', 104, 82)
('Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk', 100, 93)
('Kggcafo(EK/{(dacm(i(xg}fl(gn(jikgf', 86, 80)
('Nbbfdcj-@N*~-adfh-l-}bxci-bk-olnbc', 84, 85)


### Challenge 4

In [242]:
top_res = (None,0)
with open('4.txt') as f:
    for encrypted in f:
        res, _, _ = decrypt_using_single_byte(encrypted)
        if res[1] > top_res[1]:
            top_res = res

print(top_res)

('Now that the party is jumping\n', 15, 53)


### Challenge 5

In [263]:
### Questionable instructions again.  The space at the end of the first line is NOT to be encrypted, but the carriage return is.  However,
### the carriage return in the expected encryption should be ignored.  That is, there is no one-to-mapping between the plaintext lines
### and the encrypted lines.  Would have been clearer if the expected encryption was presented as a single line of text.

ptxt = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
key = "ICE"
expected = \
    "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63" \
    "343c2a26226324272765272a282b2f20430a652e2c652a3124333" \
    "a653e2b2027630c692b20283165286326302e27282f"

print(ptxt)

ptxt_bytes = [ord(c) for c in ptxt ]
key_bytes = [ord(c) for c in key]

def repeated_key_xor(ptxt_bytes, key_bytes):
    ctxt_bytes = []
    key_i = 0
    for pbyte in ptxt_bytes:
        ctxt_bytes.append(pbyte ^ key_bytes[key_i])
        key_i = (key_i + 1) % len(key_bytes)

    out = ''
    for cb in ctxt_bytes:
        out += f'{cb:02x}'
    return out

out = repeated_key_xor(ptxt_bytes, key_bytes)
assert expected == out, out

Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal


### Challenge 6

In [231]:
def hamming_dist(s1:str, s2:str) -> int:
    assert len(s1) == len(s2), f'inputs must have equal length.  {s1}: {len(s1)=}, {s2}: {len(s2)=}'
    
    s_bytes = []
    s_bits = []
    for i, s in enumerate([s1, s2]):
        if type(s) == str:
            s_bytes.append([ord(c) for c in s])
        else:
            s_bytes.append(s)
        s_bits.append(
            ''.join([f'{int_val:08b}' for int_val in s_bytes[i]])
        )
        #print(s_bytes[0])
    
    #print(s_bits[0])

    return sum(c1 != c2 for c1,c2 in zip(s_bits[0], s_bits[1]))
assert 37 == hamming_dist('this is a test','wokka wokka!!!')
assert 37 == hamming_dist(b'this is a test',b'wokka wokka!!!')

In [232]:
import base64
with open('6.txt') as f:
    b64 = ''.join(f.read().split('\n'))
data = base64.b64decode(b64)

In [272]:
type(data), len(data)

(bytes, 2876)

In [283]:
# Find the probable KEYSIZE

best = []
for KEYSIZE in range(2,41):
    block1 = data[0:KEYSIZE]
    block2 = data[KEYSIZE:KEYSIZE*2]
    dist = hamming_dist(block1, block2)/KEYSIZE
    best.append(
        (KEYSIZE, dist)
    )
    best.sort(key=lambda i: i[1])
    
best

[(5, 1.2),
 (3, 2.0),
 (2, 2.5),
 (13, 2.5384615384615383),
 (11, 2.6363636363636362),
 (20, 2.7),
 (18, 2.7777777777777777),
 (38, 2.8684210526315788),
 (15, 2.933333333333333),
 (17, 2.9411764705882355),
 (7, 3.0),
 (8, 3.0),
 (16, 3.0),
 (21, 3.0476190476190474),
 (31, 3.096774193548387),
 (40, 3.1),
 (37, 3.108108108108108),
 (23, 3.1739130434782608),
 (29, 3.206896551724138),
 (14, 3.2142857142857144),
 (25, 3.24),
 (12, 3.25),
 (35, 3.257142857142857),
 (33, 3.272727272727273),
 (10, 3.3),
 (39, 3.3076923076923075),
 (19, 3.3157894736842106),
 (34, 3.323529411764706),
 (24, 3.375),
 (36, 3.4166666666666665),
 (30, 3.433333333333333),
 (32, 3.4375),
 (27, 3.4814814814814814),
 (4, 3.5),
 (26, 3.5),
 (28, 3.5357142857142856),
 (9, 3.5555555555555554),
 (22, 3.727272727272727),
 (6, 4.0)]

In [273]:
def transpose_block(data_blocks, block_size):
    data_blocks_T = []
    for idx in range(block_size):
        block_T = []
        last_block = False
        for block in data_blocks:
            assert not last_block, f'only last block may have missing entries.  found non-final block with less than {block_size=} entries'
            if idx < len(block):
                block_T.append(block[idx])
            else:
                # should not be any more blocks after this one
                last_block=True
        data_blocks_T.append(block_T)
    return data_blocks_T

In [282]:

#for tup in best[:3]:
for tup in [(kz, None) for kz in range(2,40)]:
    KEYSIZE=tup[0]
    print(f'\n{KEYSIZE=}')
    
    # break into blocks of size KEYSIZE
    data_blocks = []
    for start in range(0,len(data),KEYSIZE):
        data_blocks.append(data[start:start+KEYSIZE])

    # transpose the blocks. thus each new block will be encrypted with
    # the same byte of the key
    data_blocks_T = transpose_block(data_blocks, KEYSIZE)
    #print(data_blocks[0:5])
    #print(data_blocks_T[0:5])

    # solve each block as a single-character XOR
    key = []
    for block in data_blocks_T:
        # convert to string of hex
        block_hex = ''.join( [f'{int_val:02x}' for int_val in block] )
        #print(block_hex)
        res, _, _ = decrypt_using_single_byte(block_hex)
        
        # res is the best (plaintext, plaintext's english score, key as a byte) tuple
        # add the character to the key
        key.append(res[2])
    
    # try key against data
    print(f'{key=}')
    out = repeated_key_xor(
        data,
        key)
    
    print(out[:20])
    print(bytes.fromhex(out)[:20])
    print(
        ''.join([chr(i) for i in bytes.fromhex(out)])[:20]
    )


KEYSIZE=2
key=[66, 96]
5f225d2d496f407f0d73
b'_"]-Io@\x7f\rs\x0c\\X\t\'\x7f\x0b|L.'
s\X	'|L.

KEYSIZE=3
key=[96, 96, 96]
7d227f2d6b6f627f2f73
b'}"\x7f-kob\x7f/s.\\z\t\x05\x7f)|n.'
}"-kob/s.\z	)|n.

KEYSIZE=4
key=[96, 96, 96, 96]
7d227f2d6b6f627f2f73
b'}"\x7f-kob\x7f/s.\\z\t\x05\x7f)|n.'
}"-kob/s.\z	)|n.

KEYSIZE=5
key=[96, 97, 97, 96, 96]
7d237e2d6b6f637e2f73
b'}#~-koc~/s.]{\t\x05\x7f(}n.'
}#~-koc~/s.]{	(}n.

KEYSIZE=6
key=[97, 97, 105, 96, 96, 96]
7c23762d6b6f637e2673
b'|#v-koc~&s.\\{\x08\x0c\x7f)|o/'
|#v-koc~&s.\)|o/

KEYSIZE=7
key=[97, 97, 96, 96, 96, 107, 97]
7c237f2d6b64637e2e73
b'|#\x7f-kdc~.s.\\q\x08\x04~)|n%'
|#-kdc~.s.\~)|n%

KEYSIZE=8
key=[97, 104, 96, 97, 99, 97, 105, 96]
7c2a7f2c686e6b7f2e7b
b'|*\x7f,hnk\x7f.{.]y\x08\x0c\x7f(tn/'
|*,hnk.{.](tn/

KEYSIZE=9
key=[97, 96, 105, 97, 105, 96, 104, 97, 104]
7c22762c626f6a7e2772
b'|"v,boj~\'r.U{\x00\x05w(to.'
|"v,boj~'r.U{ w(to.

KEYSIZE=10
key=[96, 97, 97, 105, 104, 99, 105, 97, 96, 100]
7d237e24636c6b7e2f77


In [292]:
key = [84, 101, 114, 109, 105, 110, 97, 116, 111, 114, 32, 88, 58, 32, 66, 114, 105, 110, 103, 32, 116, 104, 101, 32, 110, 111, 105, 115, 101]
print(f"key={''.join([chr(i) for i in key])}")
out = repeated_key_xor(data, key)
print(''.join([chr(i) for i in bytes.fromhex(out)]))


key=Terminator X: Bring the noise
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can t

In [ ]:
# TODO the hamming_dist got me nowhere near guessing a key size of 29
# TODO the single XOR solution returned "Terminator X: Bring thb noise"

### Challenge 7

In [317]:
key = 'YELLOW SUBMARINE'.encode()
print(f'{key=}')
with open('7.txt') as f:
    b64 = ''.join(f.read().split('\n'))
    data = base64.b64decode(b64)
print(f'{type(data)=}')
print(f'{len(key)=}')
print(f'{len(data)=}')
print(f'{len(data) % len(key)=}')

key=b'YELLOW SUBMARINE'
type(data)=<class 'bytes'>
len(key)=16
len(data)=2880
len(data) % len(key)=0


In [316]:
from Crypto.Cipher import AES
cipher = AES.new(key, AES.MODE_ECB)
ptxt = cipher.decrypt(data)
print(ptxt.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge 8

In [325]:
from itertools import combinations
from statistics import mean

lowest_dist = float('inf')
lowest_idx = -1
lowest_line = None
with open('8.txt') as f:
    for i,line in enumerate(f):
        data = bytes.fromhex(line.strip())
        
        # break into chunks of 16 bytes
        sz = 16
        chunks = [data[start:start+sz] for start in range(0,len(data),sz)]
        
        # find mean hamming distance between chunks.
        # If there is repeated plaintext, that will result
        # in repeated ciphertext, which is a characteristic of ECB
        mean_dist = mean(
            [hamming_dist(*tup) for tup in combinations(chunks,2)]
        )
        
        if mean_dist < lowest_dist:
            lowest_dist = mean_dist
            lowest_idx = i
            lowest_line = line
            print(lowest_dist, lowest_idx, lowest_line, '\n')
            
            
        
        

62.888888888888886 0 8a10247f90d0a05538888ad6205882196f5f6d05c21ec8dca0cb0be02c3f8b09e382963f443aa514daa501257b09a36bf8c4c392d8ca1bf4395f0d5f2542148c7e5ff22237969874bf66cb85357ef99956accf13ba1af36ca7a91a50533c4d89b7353f908c5a166774293b0bf6247391df69c87dacc4125a99ec417221b58170e633381e3847c6b1c28dda2913c011e13fc4406f8fe73bbf78e803e1d995ce4d
 

62.84444444444444 6 cc230392928a4146fe2dfea83e2567923bb2676de8b879e12cfaa09379bef6e599aec3187be62a7335b90c269247724880835747e302068a3325ed3a02feedd5c129aa8d2846db88e29ae896a8d9355483a54bbd70e1ed628c78a8eae97a3518f33b6d9c4d04ef3bf79df7cd1042eaf209d6d52a9f1f293ae3a699a3a3dbad77a9cc0d4b0b47db49397949b61ba6be52a140a3dc7dee41d106609df433d587c3
 

62.24444444444445 14 251c7b45c01ad596fcf1732283be13cf9d2c0ff8f2a5b93fbedb6a79d28acdeb54f88e24ae1c9a02cfb0856187217287df5ff63cb43595822853f609a789b3d78fffadbd891a90867b1568e2cc551d362ac257a94d91a7eb0028e6e9bb958ff7fd203b84ed085760d4c349268b635f0759f356b7efcfbbe3d9b54583c43a38590b171e3d554b67201b7f7353ceaeb1fb09

In [326]:
i

203